# Quickstart

## Work with Data

Pytorch have two ways to work with data:
1. `torch,utils.data.DataLoader` -> stores the samples and their labels
2. `torch.utils.data.Dataset` -> wraps an iterable around the Dataset

In [19]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Pytorch offers domain-specific libraries that each include datasets, the libraries such as:
1. torchtext for text
2. torchvision for vision
3. torchaudio for audio

In [20]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

we will using the `Dataset` as an arguments to `DataLoader`. This wraps an iterable over out dataset automatic batching, sampling, shuffling, and multiprocess data loading. When define batch size of 64, each element in the dataloader iterable will return a batch of 64 features and labels.

In [21]:
batch_size = 64

# create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

In [29]:
# get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [30]:
# define model 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

Dalam satu putaran pelatihan, model membuat prediksi pada dataset pelatihan (diumpankan secara bertahap), dan melakukan backpropagasi kesalahan prediksi untuk menyesuaikan parameter model.

In [32]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch %100 ==0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
        

check the model's performance against the test dataset to ensure it is learning.

In [33]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [34]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307623 [   64/60000]
loss: 2.288009 [ 6464/60000]
loss: 2.275326 [12864/60000]
loss: 2.275357 [19264/60000]
loss: 2.249946 [25664/60000]
loss: 2.236014 [32064/60000]
loss: 2.233069 [38464/60000]
loss: 2.203890 [44864/60000]
loss: 2.204450 [51264/60000]
loss: 2.171166 [57664/60000]
Test Error: 
 Accuracy: 45.0%, Avg loss: 2.165328 

Epoch 2
-------------------------------
loss: 2.174280 [   64/60000]
loss: 2.158569 [ 6464/60000]
loss: 2.105895 [12864/60000]
loss: 2.127625 [19264/60000]
loss: 2.068396 [25664/60000]
loss: 2.025928 [32064/60000]
loss: 2.049567 [38464/60000]
loss: 1.967581 [44864/60000]
loss: 1.978235 [51264/60000]
loss: 1.916560 [57664/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.907297 

Epoch 3
-------------------------------
loss: 1.931202 [   64/60000]
loss: 1.902215 [ 6464/60000]
loss: 1.787530 [12864/60000]
loss: 1.840731 [19264/60000]
loss: 1.728317 [25664/60000]
loss: 1.682321 [32064/60000]
loss: 1.707924 [38464/

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")